In [1]:
from basic_nlp import *
from json_io import *
from ml import *

C:\Program Files\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
def keyToStr(d):
    new = {}
    for key,value in d.items():
        new[" ".join(key)] = value
    return new

from re import sub

TWEET_LINK_RE = "https://t.co/(\w)+"
TWEET_HANDLE_RE = "@(\w)+"
HASHTAG_RE = "#(\w)+"

def feature(tweet):
    tweet = sub(TWEET_HANDLE_RE, "NameTOK", tweet)
    tweet = sub(TWEET_LINK_RE, "LinkTOK", tweet)
    tweet = sub(HASHTAG_RE, "", tweet)
    tokens = tokenize(tweet)
    ull = upperLowerLen(tokens)
    cases = wordCases(ull)
    tagged = pos(tokens)
    chunked = chunk(tagged)
    (tokens, postags) = tokNoNE(chunked)
    puncuationFreqDict = punctuationFeatures(tweet)
    for key,val in puncuationFreqDict.items():
        puncuationFreqDict[key]=val[2]
    suffreq = dict(freq(tokenSuffixes(tokens)))
    normSuffFreq = {}
    norm2SuffFreq = {}
    sumSuf = sum(suffreq.values())
    for key, val in suffreq.items():
        normSuffFreq[key] = val/sumSuf
        norm2SuffFreq[key] = val/len(tokens)
    
    sent = {
        'fullSent' : sentimentGrams([tokens]),
        'halfSent1' : sentimentGrams([tokens[:int(len(tokens)/2)]]),
        'halfSent2' : sentimentGrams([tokens[int(len(tokens)/2):]]),
        'thirdSent1' : sentimentGrams([tokens[:int(len(tokens)/3)]]),
        'thirdSent2' : sentimentGrams([tokens[int(len(tokens)/3):2*int(len(tokens)/3)]]),
        'thirdSent3' : sentimentGrams([tokens[2*int(len(tokens)/3):]]),
        'quartSent1' : sentimentGrams([tokens[:int(len(tokens)/4)]]),
        'quartSent2' : sentimentGrams([tokens[int(len(tokens)/4):2*int(len(tokens)/4)]]),
        'quartSent3' : sentimentGrams([tokens[2*int(len(tokens)/4):3*int(len(tokens)/4)]]),
        'quartSent4' : sentimentGrams([tokens[3*int(len(tokens)/4):]])
    }
    sentCompound = {}
    for key, val in sent.items():
        sentCompound[key+"Vader"] = val[0]['Vader']['compound'] + 1
        sentCompound[key+"LiuHu"] = val[0]['LiuHu']['compound'] + 1
    
    capFreq = capLetterFreq(ull)
    allCapsFreq = cases.count('AC')/len(cases)
    normSuffFreq = keyToStr(normSuffFreq)
    norm2SuffFreq = keyToStr(norm2SuffFreq)
    toksuff = keyToStr(dict(freq(tokenSuffixes(tokens))))
    unigrams = keyToStr(dict(freq(grams(tokens, 1))))
    bigrams = keyToStr(dict(freq(grams(tokens, 2))))
    trigrams = keyToStr(dict(freq(grams(tokens, 3))))
    unigramsPos = keyToStr(dict(freq(grams(postags, 1))))
    bigramsPos = keyToStr(dict(freq(grams(postags, 2))))
    trigramsPos = keyToStr(dict(freq(grams(postags, 3))))
    feat = {}
    feat.update(unigrams)
    feat.update(bigrams)
    feat.update(trigrams)
    feat.update(unigramsPos)
    feat.update(bigramsPos)
    feat.update(trigramsPos)
    feat.update(puncuationFreqDict)
    feat.update(toksuff)
    feat.update(normSuffFreq)
    feat.update(norm2SuffFreq)
    feat.update(sentCompound)
    feat.update({"capFreq":capFreq, "allCapsFreq":allCapsFreq})
    return feat

In [ ]:
total = 20000
trainTestCount = int(total/2)

In [ ]:
sarcasticTweets = tweet_iterate("../json/sarcastic/unique.json", key="text")
sarcasticFeats = [(feature(repr(next(sarcasticTweets))), True) for x in range(0,total)]

seriousTweets = tweet_iterate("../json/non_sarcastic/unique.json", key="text")
seriousFeats = [(feature(repr(next(seriousTweets))), False) for x in range(0,total)]

In [ ]:
t = []
a = []

In [ ]:
from random import shuffle
featTup = sarcasticFeats + seriousFeats
shuffle(featTup)
(feats, bools) = list(zip(*featTup))

In [ ]:
dv = DictVectorizer()
vec = dv.fit_transform(feats[:trainTestCount])

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
classifier = [
    LogisticRegressionCV(),
    LogisticRegression(),
    SGDClassifier(),
    LinearDiscriminantAnalysis(), 
    #SVC(),
    #GaussianProcessClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(8),
    RandomForestClassifier(16),
    RandomForestClassifier(32),
    RandomForestClassifier(64),
    #MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    BernoulliNB(),
    MultinomialNB(),
    #QuadraticDiscriminantAnalysis()
    ]
d = {}
for i, c in enumerate(classifier):
    d[str(type(c))+str(i)] = c
    print(str(i)+"\t"+str(type(c)))

m = ml(d)

In [ ]:
trainOutput = m.trainVectorizedFeatures(vec, bools[:trainTestCount])
t.append(dict([(k,v[1].total_seconds()) for k,v in trainOutput.items()]))

total = 0
for w in sorted(t[-1], key=t[-1].get, reverse=True):
    total += t[-1][w]
    print(w, t[-1][w])
print("Total: ", total)

In [ ]:
a.append(m.accuracyVectorizedFeatures(dv.transform(feats[trainTestCount:]), bools[trainTestCount:]))
for w in sorted(a[-1], key=a[-1].get, reverse=True):
    print(w, a[-1][w])